In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import glob, os, re
import matplotlib.pyplot as plt
from datetime import  datetime
from ipywidgets import widgets, interactive

In [2]:
dt = datetime.today().strftime(format='%d-%m-%Y')

### **Establish parent directory**

In [3]:
#uniqueness in directory and file names is assumed for all analyses
sars_dir = "SARS-CoV-2"
home_dir = os.getenv('HOME')
parent_dir = glob.glob(f'{home_dir}/**/{sars_dir}', recursive=True)[0]

In [4]:
cols1 = ['Sample_Name', 'Reporter', 'CT', 'Ct_Mean', 'Well_Position']

data = pd.DataFrame(columns=cols1)
data_list = []
for file in os.listdir(glob.glob(f'{parent_dir}/**/RunSheetsDB')[0]):
    if file.endswith('.xls'):
        data_list.append(file)
    pass
sheets = sorted(data_list)

In [5]:
len(np.unique(np.array(sheets)))

589

In [6]:
data.head()

,Sample_Name,Reporter,CT,Ct_Mean,Well_Position


In [7]:
' '.join('2020-10-29 15:46:45 PM EAT'.split(' ')[:-2])

'2020-10-29 15:46:45'

In [8]:
pd.to_datetime(' '.join('2020-10-29 15:46:45 PM EAT'.split(' ')[:-2]), yearfirst=True)

Timestamp('2020-10-29 15:46:45')

In [9]:
cols2 = ['Sample Name', 'Reporter', 'CT', 'Ct Mean', 'Cт', 'Cт Mean', 'Well']
cols2b = ['Sample Name', 'Reporter', 'CT', 'Ct Mean', 'Cт', 'Cт Mean', 'Well Position']

counts = 0

for sheet in sheets:
    df = pd.read_excel(f"{glob.glob(f'{parent_dir}/**/RunSheetsDB')[0]}/{sheet}")
    count2 = 0
#     print(sheet)
    try:
        for row in df.itertuples():
            count2 += 1
            if row[1] == 'Experiment Run End Time':
                expt_run_date = pd.to_datetime(' '.join(row[2].split(' ')[:-2]), yearfirst=True)
            if row[1] == 'Well' and row[2] != 'Well Position':
                df = pd.read_excel(f"{glob.glob(f'{parent_dir}/**/RunSheetsDB')[0]}/{sheet}", header=count2)
                cols3 = list(set(df.columns) & set(cols2))
    #             print(cols3)
                df2 = df[sorted(cols3)]
                df2.columns = sorted(cols1)
                df3 = df2.assign(Dt_Run=expt_run_date)
                data = data.append(df3, ignore_index=True)
                counts += 1
                break
            if row[1] == 'Well' and row[2] == 'Well Position':
                df = pd.read_excel(f"{glob.glob(f'{parent_dir}/**/RunSheetsDB')[0]}/{sheet}", 'Results', header=count2)
                cols3 = list(set(df.columns) & set(cols2b))
    #             print(cols3)
    #             print(list(df.columns))
                df2 = df[sorted(cols3)]
                df2.columns = sorted(cols1)
                df3 = df2.assign(Dt_Run=expt_run_date)
                data = data.append(df3, ignore_index=True)
                counts += 1
                break
    except:
        print(sheet)
        pass

In [10]:
print(f'number of sheets in folder: {len(sheets)}')
print(f'data rows: {data.shape[0]}')
print(f'sheets processed: {counts}')

number of sheets in folder: 589
data rows: 112221
sheets processed: 589


In [11]:
data.Sample_Name.value_counts()

PC              1780
NC              1779
NTC               87
COVC22959         72
Std1_2 x10^4      49
                ... 
COVC 17861         3
COVC 16743         3
COVC 9970          3
COVC8669           2
covc003            1
Name: Sample_Name, Length: 28193, dtype: int64

In [12]:
data.info()#.nunique()#.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112221 entries, 0 to 112220
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Sample_Name    110682 non-null  object        
 1   Reporter       110936 non-null  object        
 2   CT             110207 non-null  object        
 3   Ct_Mean        55956 non-null   float64       
 4   Well_Position  112221 non-null  object        
 5   Dt_Run         110833 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 5.1+ MB


In [13]:
data.sort_values('Dt_Run', ascending=False).to_excel(f"{glob.glob(f'{parent_dir}/Outputs')[0]}/runsheet-raw-cts.xlsx", index=False, float_format='%.1f')

In [14]:
# undetermined_ids = list(data[data['CT'] == 'Undetermined']['Sample_Name'].unique())#.astype(list)

In [15]:
data2 = data[data['CT'] != 'Undetermined']

In [16]:
data3 = data2[data2['Reporter'].isin(['FAM', 'VIC', 'ABY', 'TAMRA']) == True]
dataqc = data2[data2['Reporter'].isin(['CY5', 'FAM', 'VIC']) == True]

In [17]:
data1 = data3[data3['Sample_Name'].isin(['PC', 'NC']) != True]

In [18]:
dataCtrl = dataqc[dataqc['Sample_Name'].isin(['PC', 'NC']) == True]
(dataCtrl.to_excel(f"{glob.glob(f'{parent_dir}/**/Outputs', recursive=True)[0]}/pc-nc_QC-QA_{dt}.xlsx"
                     , index=False, na_rep='NA', float_format='%.1f'))

In [19]:
dataCtrl.Sample_Name.value_counts()

PC    1531
NC     546
Name: Sample_Name, dtype: int64

In [20]:
data4 = data1[data1.duplicated('Sample_Name', False) == True]

In [21]:
df_qc_dup = data4.groupby('Sample_Name').count()

In [39]:
mask = df_qc_dup.Reporter > 3
mask1 = df_qc_dup.Reporter % 3 == 0
df_qc_dup1 = df_qc_dup[mask & mask1 == True]
# df_qc_dup1.to_excel(f"{glob.glob(f'{parent_dir}/Outputs')[0]}/runsheet-duplicate_qc.xlsx", index=True, float_format='%.1f')
df_qc_dup2 = (data4[data4.Sample_Name.isin(df_qc_dup1.index) & data4.Sample_Name.str.contains('COV')]
.sort_values(['Sample_Name', 'Dt_Run'])
.drop_duplicates(['Sample_Name', 'Dt_Run'], keep='first'))
(df_qc_dup2[df_qc_dup2.duplicated('Sample_Name',keep=False)]
.to_excel(f"{glob.glob(f'{parent_dir}/Outputs')[0]}/runsheet-duplicates-qc-{dt}.xlsx", index=False))

In [23]:
data4.head(2)

,Sample_Name,Reporter,CT,Ct_Mean,Well_Position,Dt_Run
126,COVC 13583,FAM,37.328472,37.328472,D7,2020-10-12 15:33:47
127,COVC 13583,VIC,42.683975,42.683975,D7,2020-10-12 15:33:47


In [24]:
def get_upper_alpha_seq():
    letters = []
    for c in range(65, 91):
        letters.append(chr(c))
    return letters

In [25]:
def cov_rename(x):
    try:
        if 'COV' not in x and '-R' in x:
            x = x.replace('-R', '')
        if 'COV' in x and 'COVC' not in x and 'COVM' not in x and 'COVE' not in x:
            x = x.replace('COV', 'COVC')
        int(x)
        x = str(x).replace(str(x), f'COVC{str(x)}')
    except (ValueError, AttributeError, TypeError): x = str(x).replace(' ', '')
    return x.replace('COVC_', 'COVC0')

In [26]:
def cov_names(x):
    cov_id = str(x).upper().replace(' ', '').rstrip(f'#-_{get_upper_alpha_seq()}')
    if len(cov_id) == 8:
        return cov_id.replace('COVC', 'COVC0')
    elif len(cov_id) == 7:
        return cov_id.replace('COVC', 'COVC00')
    elif len(cov_id) == 6:
        return cov_id.replace('COVC', 'COVC000')
    elif len(cov_id) == 5:
        return cov_id.replace('COVC', 'COVC0000')
    return cov_id

In [27]:
data4['Reporter'].unique()

array(['FAM', 'VIC', 'ABY', 'TAMRA'], dtype=object)

In [28]:
data5 = data4.assign(Sample_Name=data4['Sample_Name'].apply(lambda x: cov_names(cov_rename(x))))

In [29]:
data5.head()

,Sample_Name,Reporter,CT,Ct_Mean,Well_Position,Dt_Run
126,COVC13583,FAM,37.328472,37.328472,D7,2020-10-12 15:33:47
127,COVC13583,VIC,42.683975,42.683975,D7,2020-10-12 15:33:47
135,COVC13586,FAM,35.068436,35.068436,D10,2020-10-12 15:33:47
136,COVC13586,VIC,39.385445,39.385445,D10,2020-10-12 15:33:47
186,COVC13603,FAM,27.376181,27.376181,F3,2020-10-12 15:33:47


In [30]:
data6 = (data5.sort_values(['Sample_Name', 'Reporter', 'Dt_Run'])
 .drop_duplicates(['Sample_Name', 'Reporter'], keep='last', inplace=False))

In [31]:
data6#[data6['Sample_Name'].str.contains('COVC') == False]#['Dt_Run'].unique()

,Sample_Name,Reporter,CT,Ct_Mean,Well_Position,Dt_Run
42995,,ABY,22.004671,22.004671,H12,NaT
111951,,FAM,NaN,NaN,A7,NaT
111952,,VIC,NaN,NaN,A7,NaT
107743,A1,FAM,33.489597,31.697639,D1,2021-06-30 18:48:52
107744,A1,VIC,36.625538,34.432182,D1,2021-06-30 18:48:52
...,...,...,...,...,...,...
35512,SEWAGE2,VIC,37.116642,37.116642,H2,2021-04-08 15:47:49
32792,STD1_2X10^4,FAM,23.268881,24.886887,G1,2021-04-07 15:53:47
32780,STD3_2X10^2,FAM,31.191376,31.178442,F3,2021-04-07 15:53:47
32783,STD4_2X10^1,FAM,34.213322,33.965820,F4,2021-04-07 15:53:47


In [32]:
data7 = data6.sort_values(['Sample_Name']).groupby('Sample_Name').mean()

In [33]:
data7.head()

,Ct_Mean
Sample_Name,
,22.004671
A1,33.064911
C3,33.114729
COVC00137,37.646282
COVC00154,35.240269


In [34]:
data8 = data6.sort_values(['Sample_Name', 'Dt_Run']).drop_duplicates('Sample_Name', keep='last')#.to_frame()
data8

,Sample_Name,Reporter,CT,Ct_Mean,Well_Position,Dt_Run
111952,,VIC,NaN,NaN,A7,NaT
107744,A1,VIC,36.625538,34.432182,D1,2021-06-30 18:48:52
107750,C3,VIC,35.654247,34.974087,D3,2021-06-30 18:48:52
14459,COVC00137,VIC,39.514957,39.514957,D9,2020-06-03 17:04:05
14510,COVC00154,VIC,36.905937,36.905937,F2,2020-06-03 17:04:05
...,...,...,...,...,...,...
35512,SEWAGE2,VIC,37.116642,37.116642,H2,2021-04-08 15:47:49
32792,STD1_2X10^4,FAM,23.268881,24.886887,G1,2021-04-07 15:53:47
32780,STD3_2X10^2,FAM,31.191376,31.178442,F3,2021-04-07 15:53:47
32783,STD4_2X10^1,FAM,34.213322,33.965820,F4,2021-04-07 15:53:47


In [35]:
data9 = data8.assign(Well_Position=data8.Well_Position.apply(lambda x: '0'.join(list(x)) if len(x.strip()) == 2 else x))

In [36]:
data10 = data9[['Sample_Name', 'Well_Position', 'Dt_Run']].merge(data7, how='right', left_on='Sample_Name', right_index=True)

In [37]:
df_cts = data10[data10['Sample_Name'].str.contains('COV') == True]

In [38]:
data9.sort_values('Dt_Run').to_excel(f"{glob.glob(f'{parent_dir}/Outputs')[0]}/runsheet-cts.xlsx", index=False, float_format='%.1f')